In [50]:
import cv2, numpy as np

door = cv2.imread(r"C:\Users\cbamm\Downloads\frame_0000.png") # reads door image from files
door = cv2.resize(door, (int(door.shape[1]*0.4), int(door.shape[0]*0.4))) # resizes door image from files

decal = cv2.imread(r"C:\Users\cbamm\Downloads\napa1.png") # reads decal image from files
decal = cv2.resize(decal, (int(decal.shape[1]*0.4), int(decal.shape[0]*0.4))) # resizes decal image from files

# door.shape[1] = width
# door.shape[0] = height

In [51]:
points = []

def click_event(event, x, y, flags, param): # function for clicking 4 corners manually of the target area
    if event == cv2.EVENT_LBUTTONDOWN and len(points) < 4:
        points.append((x, y))
        cv2.circle(door, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow("Select 4 Corners", door)

cv2.imshow("Select 4 Corners", door)
cv2.setMouseCallback("Select 4 Corners", click_event)
cv2.waitKey(0)
cv2.destroyAllWindows()

dst_pts = np.float32(points)
print("Selected points:", dst_pts)

Selected points: []


In [52]:
# decal.shape returns a tuple like (height, width, channels), and [:2] slices out just the first two values
h, w = decal.shape[:2] # extracts h and w from decal image file. slices out first two values h, w

src_pts = np.float32([[0, 0], [w, 0], [w, h], [0, h]])  # defines the corner coords of the decal image. source points
# Top-left: (0, 0), Top-right: (w, 0), Bottom-right: (w, h), Bottom-left: (0, h) 

# clicked door points. destination points
dst_pts = np.float32([
    [133, 386],   # TL
    [278, 379],   # TR
    [280, 481],   # BR
    [133, 484]    # BL
])

In [53]:
# perspective transformation to warp "decal" onto "door"

H = cv2.getPerspectiveTransform(src_pts, dst_pts) # computes a 3x3 - perspective transformation matrix H
# H defines how to warp the source images to match the destination shape

warped = cv2.warpPerspective(decal, H, (door.shape[1], door.shape[0]))

# decal = image to be warped
# H = transformation matrix
# door.shape[1] = width
# door.shape[0] = height

cv2.imshow("Warped Decal", warped) # displays the image "warped" titled "Warped Decal"
cv2.waitKey(0) # waits for key press before continuing
cv2.destroyAllWindows() # closes all OpenCV windows after you are done viewing

In [41]:
print(H)
# Top row [a, b, c]: Controls horizontal scaling, rotation, and translation.
# Middle row [d, e, f]: Controls vertical scaling, rotation, and translation.
# Bottom row [g, h, i]: Controls perspective distortion (how lines converge).

[[ 8.93274667e-01 -1.56564565e-02  1.33000000e+02]
 [-1.46722667e-01  8.25907508e-01  3.86000000e+02]
 [-2.64000000e-04 -1.17717718e-04  1.00000000e+00]]


In [57]:
gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY) # converts warped image to grayscale to make it easier to create a mask

_, mask = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY) # creates a binary mask from the grayscale image

fg = cv2.bitwise_and(warped, warped, mask=mask)
bg = cv2.bitwise_and(door, door, mask=cv2.bitwise_not(mask))
final = cv2.add(bg, fg)

cv2.imshow("Projected Decal on Door", final)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [59]:
# cv2.imwrite(r"C:\Users\cbamm\Downloads\decal_projected.png", final) # saves image

In [55]:
cv2.imshow("gray", gray)
cv2.waitKey(0)
cv2.imshow("mask", mask)
cv2.waitKey(0)
cv2.destroyAllWindows()